# 初始化

In [ ]:
import os
import time
import threading
from http.server import HTTPServer, SimpleHTTPRequestHandler
from local_packages import *
from dotenv import load_dotenv
from queue import Queue
import concurrent.futures
import random
import json
import openai
from concurrent.futures import ThreadPoolExecutor, as_completed

# 加载环境变量
dotenv_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path)
# 设置项目根目录和图片目录
project_root = os.path.dirname(dotenv_path)

service_type = 'kimi'

def initialize_service(service_type):
    if service_type in ['zhipu', None]:
        version = 'glm-3-turbo'
        #'glm-4' 'glm-4v' 'glm-3-turbo'
        service = GLMService(version)
    elif service_type in ['kimi']:
        version = '8k'
        #'8k'1M/12￥ '32k'1M/24￥ '128k'1M/60￥
        service = KimiService(version)
    elif service_type in ['sensetime']:
        version = 'SenseChat-5'
        #SenseChat SenseChat-32K SenseChat-128K SenseChat-Turbo SenseChat-FunctionCall
        service = SenseService(version=version)
    else:
        raise ValueError('未知的服务类型')
    
    return service

service = initialize_service(service_type)

js=JSProcessor()

class ParseError(Exception):
    def __init__(self, code, message="解析失败"):
        self.code = code
        self.message = message
        super().__init__(self.message)

## 设置工作目录
期待输出的json能够在这个目录下遵循以下结构：
- /root_folder
    - raw_data.csv
    - raw_data.json
    - /step_1_results
        - /step_1_processed_data
            - result_data.json
        - /step_1_unprocessed_data
            - remain_data.json
    - /step_2_results
        - /step_2_unprocessed_data
        - /step_2_mappings
- model_path

### 设置地址变量：

In [ ]:
import os

root_folder='Task5_News_Analyser'

step_1_processed_path=os.path.join(root_folder, 'step_1_results', 'step_1_processed_data')
step_1_unprocessed_path=os.path.join(root_folder, 'step_1_results', 'step_1_unprocessed_data')
step_2_processed_path=os.path.join(root_folder, 'step_2_results', 'step_2_processed_books')
step_2_mapping_path=os.path.join(root_folder, 'step_2_results', 'step_2_mappings')

# 模型和分词器的本地路径
model_path = os.getenv('MODEL_PATH', None)

#创建区：
# 创建step_1_processed_path目录
os.makedirs(step_1_processed_path, exist_ok=True)

# 创建step_1_unprocessed_path目录
os.makedirs(step_1_unprocessed_path, exist_ok=True)

# 创建step_2_processed_path目录
os.makedirs(step_2_processed_path, exist_ok=True)

# 创建step_2_mapping_path目录
os.makedirs(step_2_mapping_path, exist_ok=True)

# 转换原始csv为json

In [ ]:
import pandas as pd
def read_csv_and_convert_to_dict(file_path):
    try:
        # 读取CSV文件
        df = pd.read_csv(file_path)
        # 初始化一个空字典用来存储所有数据
        df = df.applymap(lambda x: None if pd.isna(x) else x)
        all_data = {}
        # 遍历DataFrame的每一行
        for index, row in df.iterrows():
            # 生成每一行的字典，包括前17列和序号
            row_dict = row[:17].to_dict()
            row_dict['序号'] = index + 1  # 添加序号字段，从1开始
            # 将这个行字典添加到总字典中，用序号作为键
            all_data[f'行{index + 1}'] = row_dict
        # 打印整个字典
        print(all_data)
        return all_data
    except Exception as e:
        print("读取CSV文件并转换为字典时出错:", e)

In [ ]:
csv_path=os.path.join(root_folder,'raw_data.csv')
json_path=os.path.join(root_folder,'raw_data.json')

## **执行1_1**

In [ ]:
raw_dict=read_csv_and_convert_to_dict(csv_path)
js.write_json(raw_dict,json_path)

# 逐条多线程处理

In [ ]:
from threading import Thread, Lock
from queue import Queue, Empty
import traceback

# 签名注释用
from typing import Generator

# 定义一个线程安全的字典和锁
result_dict = {}
lock = Lock()

#原子任务处理函数
def parse_single_file(original_single_dict: dict, index:str, key_word: str) -> dict:
    """
    解析单个文件，并返回解析结果字典。

    参数:
        original_single_dict (dict): 包含原始数据的字典。
        target_keys (list): 目标键列表。

    返回:
        dict: 包含解析结果的字典，格式为 {书名: [知识点1, 知识点2, ...]}。

    Raises:
        ParseError: 当解析失败时引发自定义1001异常
    """
    result = {}
    comment=original_single_dict['微博正文']
    # 构建提示信息
    prompt = f'''
    以下内容是{key_word}领域的一则微博评论{comment}，我要求你输出一个json字典，包含两个键值对，其键分别为“关键词”和“情感态度”，必须是如下结构：
    {{
    "关键词":[key_wd1, key_wd2, ...]
    "情感态度":positive/negative/neutral
    "激发情感的原因":...
    }}
    请注意，关键词对应的必须是列表，而情感态度对应的必须是以上三者中的一个；激发情感的原因对应一段中文解释
    注意：务必精炼，省略一切无关内容
    '''
    try:
        msg = service.ask_once(prompt)
        if msg:
            parse_success = js.parse_dict(msg)
            if parse_success:
                print(f"成功解析：{index}")
                result = parse_success
                with lock:
                    result_dict[index] = result  # 将结果存入共享字典
                print(result)
            else:
                # 解析失败，抛出异常
                error_message = f"解析失败：书名 {index}"
                error_code = 1001  # 自定义错误代码
                raise ParseError(error_code, error_message)
    except ParseError as e:
        # 如果是解析失败的异常，直接向上抛出，不做处理
        raise e
    except Exception as e:
        if 'Error code: 400' in str(e):
            error_code = 400
            error_message ="发生400错误，跳过当前处理"
            return None
        elif 'Error code: 429' in str(e):
            error_code = 429
            error_message ="发生429错误，等待30秒后继续执行"
        else:
            error_code = 1000
            error_message =f"发生未知错误{e}"
        raise ParseError(error_code, error_message)
        
    return result


#—————————————————————————————————————————————————————————————————————————————————————
#多线程处理部分
#—————————————————————————————————————————————————————————————————————————————————————

# 工作线程函数
def worker(task_queue):
    while True:
        try:
            original_single_dict, index = task_queue.get(block=False)
            parse_single_file(original_single_dict, index, '疫情健康码')
            task_queue.task_done()
        except Empty:
            break
        except Exception as e:
            print(f"未知错误: {str(e)}")
            traceback.print_exc()
            task_queue.task_done()

# 多线程处理函数
def multi_thread_processing(data_dict, num_threads=5):
    task_queue = Queue()

    # 将任务加入队列
    for index, single_dict in data_dict.items():
        task_queue.put((single_dict, index))

    # 创建并启动线程
    threads = []
    for _ in range(num_threads):
        thread = Thread(target=worker, args=(task_queue,))
        thread.start()
        threads.append(thread)

    # 等待队列清空
    task_queue.join()

    # 等待所有线程完成
    for thread in threads:
        thread.join()

    return result_dict  # 返回最终的结果字典

# **主执行块1_2**

In [ ]:
data_dict = js.read_json(json_path)
# 调用多线程处理函数，并打印结果
final_results = multi_thread_processing(data_dict, num_threads=5)
print(final_results)

# 结果格式整理

In [ ]:
def merge_dicts(dict1, dict2):
    """
    合并两个字典，其中相同键的值（假设也是字典）也会合并。

    参数:
        dict1 (dict): 第一个字典。
        dict2 (dict): 第二个字典。

    返回:
        dict: 合并后的字典。
    """
    # 创建一个新字典来存储合并后的结果
    merged_dict = {}
    
    # 获取所有键的集合
    all_keys = set(dict1.keys()) | set(dict2.keys())
    
    # 遍历所有键，合并值
    for key in all_keys:
        if key in dict1 and key in dict2:
            # 如果两个字典中都有这个键，合并这两个值（假设值也是字典）
            merged_dict[key] = {**dict1[key], **dict2[key]}
        elif key in dict1:
            # 只在第一个字典中有这个键
            merged_dict[key] = dict1[key]
        else:
            # 只在第二个字典中有这个键
            merged_dict[key] = dict2[key]
    
    return merged_dict

def sort_dict(data_dict):
    sorted_keys = sorted(data_dict.keys(), key=lambda x: int(x[1:]))  # '行1', '行2', ... '行10'
    
    # 创建有序的字典，按照键的排序
    sorted_dict = {key: data_dict[key] for key in sorted_keys}
    return sorted_dict

def rename_keys(data_dict):
    """
    将原始字典的键转换为整数，并以这些整数作为新字典的键。

    参数:
        data_dict (dict): 原始的大字典，其中键是 '行n' 形式的字符串，n 为正整数。

    返回:
        dict: 包含新键的字典。
    """
    new_dict = {}
    for key, value in data_dict.items():
        row_number = int(key[1:])  # 获取键中的行号，去掉 '行' 后转换为整数
        new_dict[row_number] = value
    
    return new_dict

import pandas as pd

def dict_to_excel(data_dict, filename):
    """
    将包含字典的大字典转换为Excel文件，先按键名排序。

    参数:
        data_dict (dict): 需要转换的大字典，其中每个元素都是一个小字典。
        filename (str): 要保存的Excel文件的名称。
    """
    # 对字典键进行排序，假设键的形式是 '行' 后跟一个数字
    sorted_keys = sorted(data_dict.keys(), key=lambda x: int(x[1:]))  # '行1', '行2', ... '行10'
    
    # 创建有序的字典，按照键的排序
    sorted_dict = {key: data_dict[key] for key in sorted_keys}
    
    # 将有序字典转换为DataFrame
    df = pd.DataFrame.from_dict(sorted_dict, orient='index')
    
    # 将DataFrame保存为Excel文件
    df.to_excel(filename, engine='openpyxl')


## **执行文件保存1_3**

In [ ]:
save_path=os.path.join(step_1_processed_path,'result.json')
merged_result=merge_dicts(data_dict,final_results)
sorted_dict=sort_dict(merged_result)
rename_dict=rename_keys(sorted_dict)
js.write_json(rename_dict,save_path)
save_excel_file=os.path.join(step_1_processed_path,'result.xlsx')
dict_to_excel(sorted_dict,'result.xlsx')

# 二、期待对数据结构做如下处理：
- 建立序号为键，字典键值对为关键词 key:embedding 的字典

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch

def load_model(model_path):
    """
    从给定的路径加载模型和分词器。

    参数:
        model_path (str): 模型的路径。

    返回:
        tuple: 包含加载的模型和分词器。
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.eval()  # 确保模型处于评估模式
    return model, tokenizer

In [7]:
pip install torch

In [ ]:
file_path=os.path.join(step_1_processed_path,'result.json')
step_2_dict=js.read_json(file_path)
for key, item in step_2_dict:
    item['关键词']

# 建立包含embedding的映射

In [ ]:
import json
import time
from transformers import AutoModel, AutoTokenizer
import torch

def embed_and_export_dict_batch(given_dict, model_path, json_file_path, batch_size=32, stats_interval=5):
    # 加载模型和分词器
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.eval()  # 确保模型处于评估模式

    start_time = time.time()

    # 准备数据，以(key, item)的形式迭代
    items = list(given_dict.items())
    total_keys = len(items)
    batches_processed = 0

    for i in range(0, total_keys, batch_size):
        batch_items = items[i:i+batch_size]
        batch_key_names = [item[1]["key_name"] for item in batch_items]
        inputs = tokenizer(batch_key_names, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()

        # 将嵌入向量添加回原字典
        for j, (key, _) in enumerate(batch_items):
            given_dict[key]["embedding"] = embeddings[j].tolist()

        batches_processed += 1

        # 统计并打印当前速度
        if batches_processed % stats_interval == 0 or (i + batch_size) >= total_keys:
            elapsed_time = time.time() - start_time
            keys_processed = min((batches_processed) * batch_size, total_keys)
            print(f"已处理 {keys_processed}/{total_keys} 个key_name，耗时 {elapsed_time:.2f}秒，速度：{keys_processed / elapsed_time:.2f}个key_name/秒")

    # 全部处理完毕后一次性保存到JSON文件
    with open(json_file_path, 'w', encoding='utf-8') as file:
        json.dump(given_dict, file, ensure_ascii=False, indent=4)

    print("完成，已将更新后的字典导出到指定的JSON文件中。")


# **执行块3_3**
在这里我们得到包含着embedding的键

In [ ]:
given_dict=mapping_dict
json_file_path=os.path.join(step_2_mapping_path,'mapping_embedding_dict.json')
embed_and_export_dict_batch(given_dict,model_path,json_file_path,batch_size=64)

# 逐层比对融合
还回末端字段

In [ ]:
def add_null_values_back(original_dict, given_dict, current_path=[], export_folder=None):
    """
    递归地向给定字典中添加原本为null的键值对。
    
    :param original_dict: 原始的字典。
    :param given_dict: 经过处理的字典，需要在这个字典中添加值。
    :param current_path: 当前的遍历路径，用于构建给定字典中的键。
    :param export_folder: 导出文件夹的路径。
    """
    for key, value in original_dict.items():
        # 更新当前路径
        new_path = current_path + [key]
        
        if isinstance(value, dict):
            # 检查是否是最底层的dict
            if all(v is None for v in value.values()):
                # 如果是最底层的dict，构建在given_dict中对应的键
                given_dict_key = "_".join(new_path)
                # 在given_dict中找到对应的项并添加"value"键
                given_dict_item = given_dict.get(given_dict_key)
                if given_dict_item is not None:
                    given_dict_item['value'] = {sub_key: None for sub_key in value}
            else:
                # 如果不是最底层的dict，继续递归遍历
                add_null_values_back(value, given_dict, new_path, export_folder=None)
                
    # 导出结果到文件
    if export_folder:
        export_file_path = os.path.join(export_folder, "mapping_with_embedding_value.json")
        with open(export_file_path, 'w', encoding='utf-8') as f:
            json.dump(given_dict, f, ensure_ascii=False, indent=4)

### **执行块3_4** 存入指定文件'mapping_embedding_dict.json'

In [ ]:
step_2_processed_json_path=os.path.join(step_2_processed_path,'transformed_results.json')
json_file_path=os.path.join(step_2_mapping_path,'mapping_embedding_dict.json')
original_dict=js.read_json(step_2_processed_json_path)
given_dict=js.read_json(json_file_path)
export_folder=step_2_mapping_path
# 示例使用
add_null_values_back(original_dict, given_dict,export_folder=export_folder)

# 逐层深入，迭代构建并聚合

In [ ]:
import numpy as np

def cosine_similarity_matrix(vectors):
    """构建余弦相似度矩阵"""
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    norms[norms == 0] = 1  # 避免除以零
    normalized_vectors = vectors / norms
    similarity_matrix = np.dot(normalized_vectors, normalized_vectors.T)
    return similarity_matrix

def build_mapping_relation(data, depth, similarity_threshold=0.8):
    groups = {}
    # 分组
    for key, item in data.items():
        if item['depth'] == depth:  # 只处理depth=3的键
            prefix = "_".join(key.split("_")[:3])
            groups.setdefault(prefix, []).append(key)

    # 初始化映射关系为字典
    mapping_relation = {}

    for group, keys in groups.items():
        if len(keys) < 2:  # 单个键无需比较
            continue
        # 构建矩阵
        vectors = np.array([data[key]['embedding'] for key in keys])
        # 计算余弦相似度矩阵
        sim_matrix = cosine_similarity_matrix(vectors)
        
        for i, key in enumerate(keys):
            # 找到与当前键相似度超过阈值的所有键
            sim_indices = np.where(sim_matrix[i] > similarity_threshold)[0]
            sim_indices = sim_indices[sim_indices != i]  # 排除自己
            if len(sim_indices) == 0:
                continue  # 没有超过阈值的相似键，跳过

            # 获取相似度超过阈值的键列表
            similar_keys = [keys[j] for j in sim_indices]
            # 检查当前键是否已经作为其他键的相似项被并入
            if not any(key in v for v in mapping_relation.values()):
                mapping_relation[key] = similar_keys

    return mapping_relation


In [ ]:
def remove_embeddings(data,depth):
    """
    移除depth小于3的键的embedding属性。
    """
    for key, item in data.items():
        if item['depth'] < depth and 'embedding' in item:
            del item['embedding']

def merge_dicts(data, mapping_relation):
    """
    根据并射集合并字典项。
    """
    for target_key, source_keys in mapping_relation.items():
        for source_key in source_keys:
            source_item = data.get(source_key)
            target_item = data.get(target_key)
            if source_item and target_item:
                if 'value' in target_item:
                    target_item['value'].update(source_item.get('value', {}))
                else:
                    target_item['value'] = source_item.get('value', {})

def update_keys(data, mapping_relation):
    """
    更新字典中的键路径。
    """
    # 反向构建路径替换映射
    replacement_map = {source_key: target_key for target_key, source_keys in mapping_relation.items() for source_key in source_keys}
    # 更新数据
    new_data = {}
    for key, item in data.items():
        # 检查当前键是否需要被替换
        new_key = replacement_map.get(key, key)
        new_data[new_key] = item
    return new_data

In [ ]:
import numpy as np

def process_data(data, start_depth=3, similarity_threshold=0.8):
    max_depth = max(item['depth'] for item in data.values())  # 获取最大深度
    for depth in range(start_depth, max_depth + 1):
        # 步骤1: 构建映射关系
        mapping_relation = build_mapping_relation(data, depth, similarity_threshold)
        
        # 步骤2: 移除embedding属性
        remove_embeddings(data, depth)
        
        # 步骤3: 合并字典项
        merge_dicts(data, mapping_relation)
        
        # 步骤4: 更新键路径
        data = update_keys(data, mapping_relation)
    for k,v in data.items():
        if 'embedding' in v:
            del v['embedding']
    
    return data

### **执行块3_5**

In [ ]:
data=js.read_json(os.path.join(step_2_mapping_path,'mapping_with_embedding_value.json'))
new_data=process_data(data)
output_path=os.path.join(step_2_mapping_path,'aggregated_dict.json')
js.write_json(new_data,output_path)

In [ ]:
output_path=os.path.join(step_2_mapping_path,'aggregated_dict.json')
data=js.read_json(output_path)

# 重建树形结构

In [ ]:
def rebuild_tree_structure(flat_dict):
    root = {}

    # 遍历每个键值对构建树
    for full_path, item in flat_dict.items():
        # 分割路径
        parts = full_path.split("_")
        current_level = root

        # 遍历路径的每一部分，逐层深入
        for part in parts[:-1]:
            # 如果当前层级还没有这个部分的键，则创建一个新的字典
            if part not in current_level:
                current_level[part] = {}
            current_level = current_level[part]

        # 对于value字段，需要特别处理
        if 'value' in item and item['value']:
            # 如果当前节点下有value，则将其作为当前节点的子节点
            current_level[parts[-1]] = {k: None for k in item['value']}
        else:
            # 如果没有value字段，或者value为空，则直接将该节点置为null
            current_level[parts[-1]] = None

    return root


In [ ]:
def rebuild_tree(dictionary):
    tree = {}
    for key, value in dictionary.items():
        keys = key.split('_')
        current_level = tree
        for k in keys:
            if k not in current_level:
                current_level[k] = {}
            current_level = current_level[k]
        # 确保value是一个字典且包含'value'键，然后更新current_level
        if isinstance(value, dict) and "value" in value:
            # 确保current_level是一个字典
            if isinstance(current_level, dict):
                current_level.update(value["value"])
            else:
                # 如果current_level不是字典，这可能是逻辑上的错误
                print(f"Unexpected type for current_level: {type(current_level)}",k)
    
    # 遍历树，将空字典替换为None
    def replace_empty_with_none(node):
        for k, v in node.items():
            if isinstance(v, dict) and not v:  # 空字典 {}
                node[k] = None
            elif isinstance(v, dict):
                replace_empty_with_none(v)
    
    replace_empty_with_none(tree)
    return tree


In [ ]:
tree = rebuild_tree(data)
js.write_json(tree,'test.json')

In [ ]:
def format_dict_to_hierarchy(dictionary, depth=0):
    result = ''
    for key, value in dictionary.items():
        result += '  ' * depth + '- ' + key + '\n'
        if isinstance(value, dict):
            result += format_dict_to_hierarchy(value, depth + 1)
    return result

def write_hierarchy_to_file(formatted_hierarchy, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(formatted_hierarchy)